In [1]:
import geopandas as gpd
import json
import ssl 
import geemap
ssl._create_default_https_context = ssl._create_unverified_context
import ee

# Trigger the authentication flow.
# Don't need to run the below as I am alreay authenticated
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [2]:
# Defining some dictionary and forrest definitions

globcover_2009 = {

    '11': {
        "name": 'Post-flooding or irrigated croplands',
        "forrest_flag": 0},
    '14': {"name": 'Rainfed croplands',
           "forrest_flag": 0},
    '20': {"name": 'Mosaic cropland (50-70%) / vegetation (grassland, shrubland, forest) (20-50%)',
           "forrest_flag": 1},
    '30': {"name": 'Mosaic vegetation (grassland, shrubland, forest) (50-70%) / cropland (20-50%)',
           "forrest_flag": 1},
    '40': {"name": 'Closed to open (>15%) broadleaved evergreen and/or semi-deciduous forest (>5m)',
           "forrest_flag": 1},
    '50': {"name": 'Closed (>40%) broadleaved deciduous forest (>5m)',
           "forrest_flag": 1},
    '60': {"name": 'Open (15-40%) broadleaved deciduous forest (>5m)',
           "forrest_flag": 1},
    '70': {"name": 'Closed (>40%) needleleaved evergreen forest (>5m)',
           "forrest_flag": 1},
    '90': {"name": 'Open (15-40%) needleleaved deciduous or evergreen forest (>5m)',
           "forrest_flag": 1},
    '100': {"name": 'Closed to open (>15%) mixed broadleaved and needleleaved forest (>5m)',
            "forrest_flag": 1},
    '110': {"name": 'Mosaic forest-shrubland (50-70%) / grassland (20-50%)',
            "forrest_flag": 1},
    '120': {"name": 'Mosaic grassland (50-70%) / forest-shrubland (20-50%)',
            "forrest_flag": 1},
    '130': {"name": 'Closed to open (>15%) shrubland (<5m)',
            "forrest_flag": 0},
    '140': {"name": 'Closed to open (>15%) grassland',
            "forrest_flag": 0},
    '150': {"name": 'Sparse (>15%) vegetation (woody vegetation, shrubs, grassland)',
            "forrest_flag": 1},
    '160': {"name": 'Closed (>40%) broadleaved forest regularly flooded - Fresh water',
            "forrest_flag": 1},
    '170': {"name": 'Closed (>40%) broadleaved semi-deciduous and/or evergreen forest regularly flooded - saline water',
            "forrest_flag": 1},
    '180': {"name": 'Closed to open (>15%) vegetation (grassland, shrubland, woody vegetation) on regularly flooded or waterlogged soil - fresh, brackish or saline water',
            "forrest_flag": 1},
    '190': {"name": 'Artificial surfaces and associated areas (urban areas >50%) GLOBCOVER 2009',
            "forrest_flag": 0},
    '200': {"name": 'Bare areas',
            "forrest_flag": 0},
    '210': {"name": 'Water bodies',
            "forrest_flag": 0},
    '220': {"name": 'Permanent snow and ice',
            "forrest_flag": 0},
    '230': {"name": 'Unclassified',
            "forrest_flag": 0}
}


copernicus_2015 = {
    '0': {"name": 'Unknown. No or not enough satellite data available.',
          "forrest_flag": 0},
    '20': {"name": 'Shrubs. Woody perennial plants with persistent and woody stems\nand without any defined main stem being less than 5 m tall. The\nshrub foliage can be either evergreen or deciduous.\n',
           "forrest_flag": 0},
    '30': {"name": 'Herbaceous vegetation. Plants without persistent stem or shoots above ground\nand lacking definite firm structure. Tree and shrub cover is less\nthan 10 %.\n',
           "forrest_flag": 0},
    '40': {"name": 'Cultivated and managed vegetation / agriculture. Lands covered with temporary crops followed by harvest\nand a bare soil period (e.g., single and multiple cropping systems).\nNote that perennial woody crops will be classified as the appropriate\nforest or shrub land cover type.\n',
           "forrest_flag": 0},
    '50': {"name": 'Urban / built up. Land covered by buildings and other man-made structures.',
           "forrest_flag": 0},
    '60': {"name": 'Bare / sparse vegetation. Lands with exposed soil, sand, or rocks and never has\nmore than 10 % vegetated cover during any time of the year.\n',
           "forrest_flag": 0},
    '70': {"name": 'Snow and ice. Lands under snow or ice cover throughout the year.',
           "forrest_flag": 0},
    '80': {"name": 'Permanent water bodies. Lakes, reservoirs, and rivers. Can be either fresh or salt-water bodies.',
           "forrest_flag": 0},
    '90': {"name": 'Herbaceous wetland. Lands with a permanent mixture of water and herbaceous\nor woody vegetation. The vegetation can be present in either salt,\nbrackish, or fresh water.\n',
           "forrest_flag": 1},
    '100': {"name": 'Moss and lichen.',
            "forrest_flag": 1},
    '111': {"name": 'Closed forest, evergreen needle leaf. Tree canopy >70 %, almost all needle leaf trees remain\ngreen all year. Canopy is never without green foliage.\n',
            "forrest_flag": 1},
    '112': {"name": 'Closed forest, evergreen broad leaf. Tree canopy >70 %, almost all broadleaf trees remain\ngreen year round. Canopy is never without green foliage.\n',
            "forrest_flag": 1},
    '113': {"name": 'Closed forest, deciduous needle leaf. Tree canopy >70 %, consists of seasonal needle leaf\ntree communities with an annual cycle of leaf-on and leaf-off\nperiods.\n',
            "forrest_flag": 1},
    '114': {"name": 'Closed forest, deciduous broad leaf. Tree canopy >70 %, consists of seasonal broadleaf\ntree communities with an annual cycle of leaf-on and leaf-off periods.\n',
            "forrest_flag": 1},
    '115': {"name": 'Closed forest, mixed.',
            "forrest_flag": 1},
    '116': {"name": 'Closed forest, not matching any of the other definitions.',
            "forrest_flag": 1},
    '121': {"name": 'Open forest, evergreen needle leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, almost all needle leaf trees remain green all year.\nCanopy is never without green foliage.\n',
            "forrest_flag": 1},
    '122': {"name": 'Open forest, evergreen broad leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, almost all broadleaf trees remain green year round.\nCanopy is never without green foliage.\n',
            "forrest_flag": 1},
    '123': {"name": 'Open forest, deciduous needle leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, consists of seasonal needle leaf tree communities with\nan annual cycle of leaf-on and leaf-off periods.\n',
            "forrest_flag": 1},
    '124': {"name": 'Open forest, deciduous broad leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, consists of seasonal broadleaf tree communities with\nan annual cycle of leaf-on and leaf-off periods.\n',
            "forrest_flag": 1},
    '125': {"name": 'Open forest, mixed.',
            "forrest_flag": 1},
    '126': {"name": 'Open forest, not matching any of the other definitions.',
            "forrest_flag": 1},
    '200': {"name": 'Oceans, seas. Can be either fresh or salt-water bodies.',
            "forrest_flag": 0},
}

copernicus_2019 = {
    '0': {
        "name": 'Unknown. No or not enough satellite data available.',
        "forrest_flag": 0},
    '20': {
        "name": 'Shrubs. Woody perennial plants with persistent and woody stems\nand without any defined main stem being less than 5 m tall. The\nshrub foliage can be either evergreen or deciduous.\n',
        "forrest_flag": 0},
    '30': {
        "name": 'Herbaceous vegetation. Plants without persistent stem or shoots above ground\nand lacking definite firm structure. Tree and shrub cover is less\nthan 10 %.\n',
        "forrest_flag": 0},
    '40': {
        "name": 'Cultivated and managed vegetation / agriculture. Lands covered with temporary crops followed by harvest\nand a bare soil period (e.g., single and multiple cropping systems).\nNote that perennial woody crops will be classified as the appropriate\nforest or shrub land cover type.\n',
        "forrest_flag": 0},
    '50': {
        "name": 'Urban / built up. Land covered by buildings and other man-made structures.',
        "forrest_flag": 0},
    '60': {
        "name": 'Bare / sparse vegetation. Lands with exposed soil, sand, or rocks and never has\nmore than 10 % vegetated cover during any time of the year.\n',
        "forrest_flag": 0},
    '70': {
        "name": 'Snow and ice. Lands under snow or ice cover throughout the year.',
        "forrest_flag": 0},
    '80': {
        "name": 'Permanent water bodies. Lakes, reservoirs, and rivers. Can be either fresh or salt-water bodies.',
        "forrest_flag": 0},
    '90': {
        "name": 'Herbaceous wetland. Lands with a permanent mixture of water and herbaceous\nor woody vegetation. The vegetation can be present in either salt,\nbrackish, or fresh water.\n',
        "forrest_flag": 1},
    '100': {
        "name": 'Moss and lichen.',
        "forrest_flag": 1},
    '111': {
        "name": 'Closed forest, evergreen needle leaf. Tree canopy >70 %, almost all needle leaf trees remain\ngreen all year. Canopy is never without green foliage.\n',
        "forrest_flag": 1},
    '112': {
        "name": 'Closed forest, evergreen broad leaf. Tree canopy >70 %, almost all broadleaf trees remain\ngreen year round. Canopy is never without green foliage.\n',
        "forrest_flag": 1},
    '113': {
        "name": 'Closed forest, deciduous needle leaf. Tree canopy >70 %, consists of seasonal needle leaf\ntree communities with an annual cycle of leaf-on and leaf-off\nperiods.\n',
        "forrest_flag": 1},
    '114': {
        "name": 'Closed forest, deciduous broad leaf. Tree canopy >70 %, consists of seasonal broadleaf\ntree communities with an annual cycle of leaf-on and leaf-off periods.\n',
        "forrest_flag": 1},
    '115': {
        "name": 'Closed forest, mixed.',
        "forrest_flag": 1},
    '116': {
        "name": 'Closed forest, not matching any of the other definitions.',
        "forrest_flag": 1},
    '121': {
        "name": 'Open forest, evergreen needle leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, almost all needle leaf trees remain green all year.\nCanopy is never without green foliage.\n',
        "forrest_flag": 1},
    '122': {
        "name": 'Open forest, evergreen broad leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, almost all broadleaf trees remain green year round.\nCanopy is never without green foliage.\n',
        "forrest_flag": 1},
    '123': {
        "name": 'Open forest, deciduous needle leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, consists of seasonal needle leaf tree communities with\nan annual cycle of leaf-on and leaf-off periods.\n',
        "forrest_flag": 1},
    '124': {
        "name": 'Open forest, deciduous broad leaf. Top layer- trees 15-70 % and second layer- mixed of shrubs\nand grassland, consists of seasonal broadleaf tree communities with\nan annual cycle of leaf-on and leaf-off periods.\n',
        "forrest_flag": 1},
    '125': {
        "name": 'Open forest, mixed.',
        "forrest_flag": 1},
    '126': {
        "name": 'Open forest, not matching any of the other definitions.',
        "forrest_flag": 1},
    '200': {
        "name": 'Oceans, seas. Can be either fresh or salt-water bodies.',
        "forrest_flag": 0},
}


esaworldcover_2021 = {
    '10': {"name": 'Tree cover', "forrest_flag": 1},
    '20': {"name": 'Shrubland', "forrest_flag": 0},
    '30': {"name": 'Grassland', "forrest_flag": 0},
    '40': {"name": 'Cropland', "forrest_flag": 0},
    '50': {"name": 'Built-up', "forrest_flag": 0},
    '60': {"name": 'Bare / sparse vegetation', "forrest_flag": 0},
    '70': {"name": 'Snow and ice', "forrest_flag": 0},
    '80': {"name": 'Permanent water bodies', "forrest_flag": 0},
    '90': {"name": 'Herbaceous wetland', "forrest_flag": 1},
    '95': {"name": 'Mangroves', "forrest_flag": 1},
    '100': {"name": 'Moss and lichen', "forrest_flag": 1},
}

In [3]:
from typing import Dict, Optional
def ee_geo_compatible(geoJSON_input: Dict):
    geoJSON_output = dict(geoJSON_input)
    try:
        if geoJSON_input["type"]=="Polygon":
            check_coords = geoJSON_input["coordinates"][0][0]
            if len(check_coords)>2:
                print("Forcefully making coordinates compatible ee.Geometry as current len>2:", check_coords)
                new_coords = [[x[0:2] for x in y] for y in geoJSON_input["coordinates"]]
                geoJSON_output["coordinates"] = new_coords
        else:
            print("currently unknown handling for type:", geoJSON_input["type"])
        return geoJSON_output
    except:
        print("Dict passed does not follow geoJSON format")
        return None

In [4]:
# Read in KML file
import geopandas as gpd
from fiona.drvsupport import supported_drivers
# Path to your KML file
kml_file_path = 'townsville_v2.kml'

# Read the KML file using geopandas
supported_drivers['KML'] = 'rw'
my_map = gpd.read_file(kml_file_path, driver='KML')
kml_geo_json=json.loads(my_map.to_json())
new_kml_geo_json = dict(kml_geo_json)
# Initiate with no features
new_kml_geo_json["features"] = []
# display("original geojson")
# display(kml_geo_json)
for feat in kml_geo_json["features"]:
    new_feat = dict(feat)
    new_feat["geometry"] = ee_geo_compatible(feat["geometry"])
    if new_feat["geometry"]:
        try:
            ee.Geometry(new_feat["geometry"])
            print("succesfully passed to ee.Geometry")
            new_kml_geo_json["features"] += [new_feat]
        except:
            print("Unable to pass into ee.Geometry asses below feature")
            print(new_feat)
            
# display("updated geoJSON")
# display(new_kml_geo_json)
geojson_fc = ee.FeatureCollection(new_kml_geo_json)
# display(geojson_fc.geometry())

Forcefully making coordinates compatible ee.Geometry as current len>2: [146.8414434142778, -19.31390190415059, 0.0]
succesfully passed to ee.Geometry


In [5]:
import pandas as pd
# Initialise the Geemap

m = geemap.Map()
poly = geojson_fc.geometry()
poly_area = poly.area().getInfo()
m.centerObject(poly, 12)
display(m)

# Standard Functions 


def class_val_dict(val_list, class_name_list):
    output_dict = {}
    if len(val_list)==len(class_name_list):
        for i in range(len(val_list)):
            output_dict[f"{val_list[i]}"]=class_name_list[i]
    return output_dict

def area_sum_by_class_func(item):
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('class')).format()
    area = ee.Number(areaDict.get('sum')).round()
    return ee.List([classNumber, area])

def display_area_by_class(dataset, poly_clip, class_name_dict = {}):
    # print(class_name_dict)
    data=pd.DataFrame()
    areaImage = ee.Image.pixelArea().addBands(dataset.clip(poly_clip))
    areas = areaImage.reduceRegion(
        reducer = ee.Reducer.sum().group(
            groupField= 1,
            groupName= 'class',
        ),
        geometry= poly,
        scale= 1,
        maxPixels= 1e10
        )
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(area_sum_by_class_func)
    result = ee.Dictionary(classAreaLists.flatten())
    tot_forrest_perc = 0
    forrest_flag = 0
    for val, area in result.getInfo().items():
        perc =  area/poly_area*100
        try:
            # print(class_name_dict)
            class_name = class_name_dict[f"{val}"]['name']
            forrest_flag = class_name_dict[f"{val}"]['forrest_flag']
            if forrest_flag:
                tot_forrest_perc += perc
        except:
            class_name = "NA"
            forrest_flag = 0
        new_row=pd.DataFrame({
            'class_name': [class_name], 
            'class_val': [val], 
            'area': [round(area, 2)],
            'perc': [round(perc, 2)], 
            'forrest_flag': forrest_flag,
            'forrest_cum_perc': [round(tot_forrest_perc,2)],
        })
        data = pd.concat([data,new_row])
            # print('class Val:', val, 'Area:', round(area, 2), 'Perc:', round(perc, 2), 'total_forrest_Perc:', round(tot_perc,2))
    if result.getInfo():
        return data.reset_index(drop=True)

# import land cover maps 2009, 2015, 2019, 2021

# 2009 GlobCover
dataset_2009 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
    'landcover'
)
visualization = {
}
m.add_ee_layer(dataset_2009.clip(poly), visualization, '2009_Landcover', shown=True)
m.add_legend(title='2009_Landcover', builtin_legend='GLOBCOVER', layer_name='2009_Landcover')
m.show_layer(name='2009_Landcover' , show=False)
# 2009 GlobCover Area 
display('## 2009 GlobCover') 
display(display_area_by_class(dataset_2009, poly, globcover_2009))

# 2015 Copernicus
visualization = {
    'bands': ['discrete_classification'],
}
dataset_2015 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015').select(
    'discrete_classification'
)
m.add_ee_layer(dataset_2015.clip(poly), visualization, '2015_Landcover', shown=False)
m.add_legend(title='2015_Landcover', builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global', layer_name='2015_Landcover')
m.show_layer(name='2015_Landcover' , show=False)
# 2015 Copernicus Area 
display('## 2015 Copernicus') 
new_dict = class_val_dict(dataset_2015.get("discrete_classification_class_values").getInfo(), dataset_2015.get("discrete_classification_class_names").getInfo())
display(display_area_by_class(dataset_2015, poly, copernicus_2015))


# 2019 Copernicus
dataset_2019 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)
visualization = {
    'bands': ['discrete_classification'],
}
m.add_ee_layer(dataset_2019.clip(poly), visualization, '2019_Landcover', shown=False)
m.add_legend(title='2019_Landcover', builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global', layer_name='2019_Landcover')
m.show_layer(name='2019_Landcover' , show=False)
# 2019 Copernicus Area 
display('## 2019 Copernicus') 
new_dict = class_val_dict(dataset_2019.get("discrete_classification_class_values").getInfo(), dataset_2019.get("discrete_classification_class_names").getInfo())

display(display_area_by_class(dataset_2019, poly, copernicus_2019))


# 2021 ESA World Cover
dataset_2021 = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map')
visualization = {
    'bands': ['Map'],
}
m.add_ee_layer(dataset_2021.clip(poly), visualization, '2021_Landcover', shown=True)
m.add_legend(title='2021_Landcover', builtin_legend='ESA_WorldCover', layer_name='2021_Landcover')
# 2021 ESA World Cover Area 
display('## 2021 ESA World Cover') 
new_dict = class_val_dict(dataset_2021.get("Map_class_values").getInfo(), dataset_2021.get("Map_class_names").getInfo())
display(display_area_by_class(dataset_2021, poly, esaworldcover_2021))


# Add historical satalite views 
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}
sat_dict = {}
for yr in ['2015','2017','2019', '2021', '2023']:
    sat_dict[f"dataset_sat_{yr}"] = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterDate(f'{yr}-01-01', f'{yr}-12-31')
        # Pre-filter to get less cloudy granules.
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
    )

    m.add_layer(sat_dict[f"dataset_sat_{yr}"].median().clip(poly), rgb_vis, f'{yr}_Satalite', shown=False)


Map(center=[-19.297110979859585, 146.84720385099328], controls=(WidgetControl(options=['position', 'transparen…

'## 2009 GlobCover'

,class_name,class_val,area,perc,forrest_flag,forrest_cum_perc
0,Mosaic forest-shrubland (50-70%) / grassland (...,110,1291073,6.89,1,6.89
1,Mosaic grassland (50-70%) / forest-shrubland (...,120,2250776,12.02,1,18.91
2,Closed to open (>15%) shrubland (<5m),130,8174867,43.65,0,18.91
3,Closed to open (>15%) grassland,140,89641,0.48,0,18.91
4,"Sparse (>15%) vegetation (woody vegetation, sh...",150,895423,4.78,1,23.69
5,Artificial surfaces and associated areas (urba...,190,2714457,14.49,0,23.69
6,Water bodies,210,374891,2.00,0,23.69
7,Closed to open (>15%) broadleaved evergreen an...,40,2803506,14.97,1,38.66
8,Open (15-40%) broadleaved deciduous forest (>5m),60,79068,0.42,1,39.08


'## 2015 Copernicus'

,class_name,class_val,area,perc,forrest_flag,forrest_cum_perc
0,"Closed forest, evergreen broad leaf. Tree cano...",112,2508822,13.39,1,13.39
1,"Closed forest, deciduous broad leaf. Tree cano...",114,1435674,7.67,1,21.06
2,"Closed forest, not matching any of the other d...",116,70885,0.38,1,21.44
3,"Open forest, evergreen broad leaf. Top layer- ...",122,35397,0.19,1,21.63
4,"Open forest, deciduous broad leaf. Top layer- ...",124,2406321,12.85,1,34.47
5,"Open forest, not matching any of the other def...",126,2463235,13.15,1,47.63
6,Shrubs. Woody perennial plants with persistent...,20,685990,3.66,0,47.63
7,"Oceans, seas. Can be either fresh or salt-wate...",200,691865,3.69,0,47.63
8,Herbaceous vegetation. Plants without persiste...,30,7640225,40.79,0,47.63
9,Cultivated and managed vegetation / agricultur...,40,283074,1.51,0,47.63


'## 2019 Copernicus'

,class_name,class_val,area,perc,forrest_flag,forrest_cum_perc
0,"Closed forest, evergreen broad leaf. Tree cano...",112,2508822,13.39,1,13.39
1,"Closed forest, deciduous broad leaf. Tree cano...",114,1435674,7.67,1,21.06
2,"Closed forest, not matching any of the other d...",116,70885,0.38,1,21.44
3,"Open forest, evergreen broad leaf. Top layer- ...",122,35397,0.19,1,21.63
4,"Open forest, deciduous broad leaf. Top layer- ...",124,2369894,12.65,1,34.28
5,"Open forest, not matching any of the other def...",126,2328007,12.43,1,46.71
6,Shrubs. Woody perennial plants with persistent...,20,634528,3.39,0,46.71
7,"Oceans, seas. Can be either fresh or salt-wate...",200,691865,3.69,0,46.71
8,Herbaceous vegetation. Plants without persiste...,30,7456730,39.81,0,46.71
9,Cultivated and managed vegetation / agricultur...,40,283074,1.51,0,46.71


'## 2021 ESA World Cover'

,class_name,class_val,area,perc,forrest_flag,forrest_cum_perc
0,Tree cover,10,2042135,10.90,1,10.90
1,Shrubland,20,5598,0.03,0,10.90
2,Grassland,30,7846433,41.89,0,10.90
3,Cropland,40,117943,0.63,0,10.90
4,Built-up,50,147392,0.79,0,10.90
5,Bare / sparse vegetation,60,2015084,10.76,0,10.90
6,Permanent water bodies,80,256416,1.37,0,10.90
7,Herbaceous wetland,90,8371,0.04,1,10.95
8,Mangroves,95,6234333,33.29,1,44.23


In [7]:
if m.draw_features:
    new_drawn_fc = ee.FeatureCollection(m.draw_features)
    new_poly = new_drawn_fc.geometry()
    new_poly_area = new_poly.area().getInfo()
    # 2021 ESA World Cover
    dataset_new = ee.ImageCollection('ESA/WorldCover/v200').first()
    visualization = {
        'bands': ['Map'],
    }
    
    # 2009 GlobCover Area 
    display('## 2009 GlobCover Drawn Poly') 
    dataset_2009 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
        'landcover'
    )
    display(display_area_by_class(dataset_2009, new_poly, globcover_2009))

    # 2021 ESA World Cover Area 
    display('## 2021 ESA World Cover') 
    
    dataset_2021 = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map')
    display(display_area_by_class(dataset_2021, new_poly, esaworldcover_2021))


'## 2009 GlobCover Drawn Poly'

,class_name,class_val,area,perc,forrest_flag,forrest_cum_perc
0,Mosaic forest-shrubland (50-70%) / grassland (...,110,427412,2.28,1,2.28
1,Mosaic grassland (50-70%) / forest-shrubland (...,120,337394,1.80,1,4.08
2,Closed to open (>15%) shrubland (<5m),130,2988407,15.96,0,4.08
3,Artificial surfaces and associated areas (urba...,190,822568,4.39,0,4.08
4,Closed to open (>15%) broadleaved evergreen an...,40,1103311,5.89,1,9.97


'## 2021 ESA World Cover'

,class_name,class_val,area,perc,forrest_flag,forrest_cum_perc
0,Tree cover,10,490462,2.62,1,2.62
1,Shrubland,20,786,0.00,0,2.62
2,Grassland,30,4402667,23.51,0,2.62
3,Cropland,40,77032,0.41,0,2.62
4,Built-up,50,43993,0.23,0,2.62
5,Bare / sparse vegetation,60,248515,1.33,0,2.62
6,Herbaceous wetland,90,718,0.00,1,2.62
7,Mangroves,95,414919,2.22,1,4.84
